# Case Study, deploying a simple API with docker

When deploying FastAPI applications a common approach is to build a Linux container image. It's normally done using Docker. You can then deploy that container image in one of a few possible ways.

## Container

Containers (mainly Linux containers) are a very lightweight way to package applications including all their dependencies and necessary files while keeping them isolated from other containers (other applications or components) in the same system.

Linux containers run using the same Linux kernel of the host (machine, virtual machine, cloud server, etc). This just means that they are very lightweight (compared to full virtual machines emulating an entire operating system).

This way, containers consume little resources, an amount comparable to running the processes directly (a virtual machine would consume much more).

Containers also have their own isolated running processes (commonly just one process), file system, and network, simplifying deployment, security, development, etc.

### Container Image

A container is run from a container image.

A container image is a static version of all the files, environment variables, and the default command/program that should be present in a container. Static here means that the container image is not running, it's not being executed, it's only the packaged files and metadata.

In contrast to a "container image" that is the stored static contents, a "container" normally refers to the running instance, the thing that is being executed.

When the container is started and running (started from a container image) it could create or change files, environment variables, etc. Those changes will exist only in that container, but would not persist in the underlying container image (would not be saved to disk).

A container image is comparable to the program file and contents, e.g. python and some file main.py.

And the container itself (in contrast to the container image) is the actual running instance of the image, comparable to a process. In fact, a container is running only when it has a process running (and normally it's only a single process). The container stops when there's no process running in it.

### Containers and Processes

A container image normally includes in its metadata the default program or command that should be run when the container is started and the parameters to be passed to that program. Very similar to what would be if it was in the command line.

When a container is started, it will run that command/program (although you can override it and make it run a different command/program).

A container is running as long as the main process (command or program) is running.

A container normally has a single process, but it's also possible to start subprocesses from the main process, and that way you will have multiple processes in the same container.

But it's not possible to have a running container without at least one running process. If the main process stops, the container stops.

## Building a docker Image for FastAPI

This will show you how to build a Docker image for FastAPI from scratch, based on the official Python image.

### Setting Up Docker

To get started, you'll need to create an account on the [Docker website](https://www.docker.com/).

Once you have your account set up, open a terminal and execute the following command:

```bash
sudo docker login -u your_username
```
Replace `your_username` with your Docker username. After running this command, you'll be prompted to enter your password.

### File Structure

We will create a file structure as follows, (this is building a python packager with name `app`)

```
├── app
│   ├── __init__.py
│   └── main.py
├── Dockerfile
└── requirements.txt
```


Create an empy file `__init__.py`


Create a `main.py` with a minimal API:

```
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def alive():
    return( {"status": "alive"})
```

Create your `requirements.txt`
You could (actually should) specify specific versions in the requirements

```
fastapi 
pydantic 
uvicorn 
```

And finally create the Dockerfile

```
# base layer, a python setup
FROM python:3.9

# setup a working directory
WORKDIR /code

# Next we will install within the docker image, the requirements file
COPY ./requirements.txt /code/requirements.txt

# We will install the requirements 
RUN pip install --no-cache-dir --upgrade -r /code/requirements.txt

# copy our application
COPY ./app /code/app

# Finally run the API server with uvicorn, each command with a a new element 
# in the list

CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "80"]
```

### Building the image 

`sudo docker build -t myapiimage .`

### Start the docker container

80:80 (insidecontainer:outsidecontainer) maps the docker port to the local machine port, so it is accessible

`docker run -d --name myapi -p 80:80 myapiimage`

Now we can access our FastAPI application in our web browser by visiting http://localhost. We will see the "status: alive" message returned by our application's root route.

### Stop the Docker Container

1. **Identify the running container**: First, we need to know the name or ID of the container you want to stop. We can get a list of all running containers using the `docker ps` command.

   ```bash
   docker ps
   ```
   <br>
   
   This will show us a list of running containers along with their ID, name, status, exposed ports, etc.

2. **Stop the container**: Once we have identified the container we wish to stop, use its name or ID in the `docker stop` command followed by the name or ID of the container.

   ```bash
   docker stop <container_name_or_ID>
   ```
   
   <br>
   
   For example, if our container name is `myfastapicontainer`, you would run:

   ```bash
   docker stop myfastapicontainer
   ```
   <br>
   Or if we have the container's ID, you would use that instead:

   ```bash
   docker stop <container_ID>
   ```
   <br>

   This command sends a SIGTERM signal to the container, asking it to gracefully shut down. If the container does not stop after a reasonable amount of time, you can use `docker kill` to force stop it.

3. **Verify the container has stopped**: We can verify that the container has stopped successfully by running `docker ps` again to see the list of running containers. The container you stopped should no longer appear in the list.

### Errors?

Use 
`docker logs myapiimage`

You can always remote the image
`docker remove myappiimage`
and build it again with new corrections

## Build a Docker Image with a Single-File FastAPI

In this case, the file structure is simpler
```
.
├── Dockerfile
├── main.py
└── requirements.txt
```

The Dockfer file will be then:

```
FROM python:3.9

WORKDIR /code

COPY ./requirements.txt /code/requirements.txt

RUN pip install --no-cache-dir --upgrade -r /code/requirements.txt

# 
COPY ./main.py /code/

# 
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]
```




## Further info

- at https://fastapi.tiangolo.com/deployment/concepts/ 
- For building bigger applications https://fastapi.tiangolo.com/tutorial/bigger-applications/ 